In [162]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [163]:
data = pd.read_csv("isolates.csv")
data.head(10)

,#Organism group,Isolate,AMR genotypes,AST phenotypes
0,Listeria monocytogenes,PDT000077416.3,"fosX=COMPLETE,lin=COMPLETE","chloramphenicol=S,clindamycin=R,erythromycin=S..."
1,Listeria monocytogenes,PDT000095192.3,"fosX=COMPLETE,lin=COMPLETE","ampicillin=S,penicillin=S"
2,Salmonella enterica,PDT000003687.3,"mdsA=COMPLETE,mdsB=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
3,Salmonella enterica,PDT000003688.4,"mdsA=COMPLETE,mdsB=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
4,Salmonella enterica,PDT000003689.4,"mdsA=COMPLETE,mdsB=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
5,Salmonella enterica,PDT000003690.3,"aph(3'')-Ib=COMPLETE,aph(6)-Id=COMPLETE,mdsA=C...","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
6,Salmonella enterica,PDT000003691.3,"mdsA=COMPLETE,mdsB=COMPLETE,tet(B)=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
7,Salmonella enterica,PDT000003692.3,"mdsA=COMPLETE,mdsB=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
8,Salmonella enterica,PDT000003693.3,"aph(3'')-Ib=COMPLETE,aph(6)-Id=COMPLETE,mdsA=C...","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
9,Salmonella enterica,PDT000003694.4,"fosA7=COMPLETE,mdsA=COMPLETE,mdsB=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."


In [164]:
def transform_dataframe(df):
    new_data = []

    for idx, row in df.iterrows():
        # split AMR genotypes and remove "=COMPLETE"
        amr_genotypes = [i.split('=')[0] for i in row['AMR genotypes'].split(',')]

        # split AST phenotypes
        ast_phenotypes = row['AST phenotypes'].split(',')

        # process each phenotype
        for pheno in ast_phenotypes:
            drug, resistance = pheno.split('=')
            if resistance == 'R':
                resistance_score = 1
            elif resistance == 'S':
                resistance_score = 0
            else:  # assuming 'I' as per your description
                resistance_score = 0.5

            # create a new row
            new_row = {'#Organism group': row['#Organism group'],
                       'Isolate': row['Isolate'],
                       'AMR genotypes': ', '.join(amr_genotypes),
                       'drug': drug,
                       'resistance': resistance_score}
            new_data.append(new_row)

    # create a new dataframe
    new_df = pd.DataFrame(new_data)

    return new_df


In [165]:
df = transform_dataframe(data)


In [166]:
df.head(10)

,#Organism group,Isolate,AMR genotypes,drug,resistance
0,Listeria monocytogenes,PDT000077416.3,"fosX, lin",chloramphenicol,0.0
1,Listeria monocytogenes,PDT000077416.3,"fosX, lin",clindamycin,1.0
2,Listeria monocytogenes,PDT000077416.3,"fosX, lin",erythromycin,0.0
3,Listeria monocytogenes,PDT000077416.3,"fosX, lin",gentamicin,0.0
4,Listeria monocytogenes,PDT000077416.3,"fosX, lin",levofloxacin,0.0
5,Listeria monocytogenes,PDT000077416.3,"fosX, lin",oxacillin,1.0
6,Listeria monocytogenes,PDT000077416.3,"fosX, lin",penicillin,0.0
7,Listeria monocytogenes,PDT000077416.3,"fosX, lin",rifampin,0.0
8,Listeria monocytogenes,PDT000077416.3,"fosX, lin",tetracycline,0.0
9,Listeria monocytogenes,PDT000077416.3,"fosX, lin",trimethoprim-sulfamethoxazole,0.0


In [167]:
df.shape

(316071, 5)

In [168]:
df.to_csv("BasicData.csv",index=False)

In [169]:
unique_all_genes = set()
unique_all_antibiotics = set()

def create_empty_gene_antibiotic_df(df):
    for idx, row in df.iterrows():
        genotypes = row['AMR genotypes'].split(', ')
        antibiotic = row['drug']
        
        unique_all_genes.update(genotypes)
        unique_all_antibiotics.add(antibiotic)
    
    gene_antibiotic_df = pd.DataFrame(index=unique_all_genes, columns=unique_all_antibiotics)
    gene_antibiotic_df = gene_antibiotic_df.fillna(-1)
    
    return gene_antibiotic_df

# Assuming your DataFrame is named 'new_dataframe'
gene_antibiotic_df = create_empty_gene_antibiotic_df(new_dataframe)
gene_antibiotic_df



,chloramphenicol,dicloxacillin,ciprofloxacin,ceftiofur,fosfomycin-glucose-6-phosphate,amoxicillin-clavulanic acid,benzylpenicillin,metronidazole,linezolid,piperacillin,...,trimethoprim-sulfamethoxazole,aztreonam,norfloxacin,neomycin,Imipenem-EDTA-PA,delafloxacin,zoliflodacin,vancomycin,ertapenem,cefiderocol
qnrB48,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
blaADC-155,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
oqxB19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
blaACT-37,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
blaTEM-19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
blaOXA-494,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
qnrB38,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
tmexD3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
tet(O),-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


***********************************************

In [170]:
def gene_per_drug(drug):
    drug_df = df[df['drug'] == drug]
    unique_genes = set()
    for genes in drug_df['AMR genotypes'].str.split(', '):
        unique_genes.update(genes)
    return drug_df, unique_genes

In [171]:
def create_gene_df(drug_df, unique_genes):
    gene_arrays = []
    labels = []

    for idx, row in drug_df.iterrows():
        gene_array = np.zeros(len(unique_genes), dtype=int)
        genes = row['AMR genotypes'].split(', ')
        for gene in genes:
            gene_index = list(unique_genes).index(gene)
            gene_array[gene_index] = 1

        gene_arrays.append(gene_array.tolist())
        labels.append(row['resistance'])

    gene_df = pd.DataFrame(gene_arrays, columns=unique_genes)
    gene_df['label'] = labels

    return gene_df


In [172]:

def Model(drug_df_for_model):
    # Assuming your DataFrame with gene features and labels is named `oxacillin_df_for_model`
    # Split the data into features (genes) and labels
    features = drug_df_for_model.drop('label', axis=1).values
    labels = drug_df_for_model['label'].values

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Define the neural network model
    model = keras.Sequential()
    model.add(keras.layers.Dense(64, activation='relu', input_shape=(features.shape[1],)))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dense(16, activation='relu'))
    model.add(keras.layers.Dense(8, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))  # Sigmoid activation for probability between 0 and 1

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mse'])

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32)

    # Evaluate the model
    loss, mse = model.evaluate(X_test, y_test)
    pred = model.predict(X_test)
    print('Loss:', loss)
    print('Mean Squared Error:', mse)
    print('len of pred: ', pred.shape, 'y_test: ', y_test.shape)
    # Make predictions
    return y_test, pred, model

In [173]:
def evaluation(y_test, predictions, drug):
    # Calculate mean squared error
    mse = mean_squared_error(y_test, predictions)

    # Calculate mean absolute error
    mae = mean_absolute_error(y_test, predictions)

    # Calculate R^2 score
    r2 = r2_score(y_test, predictions)

    
    print('Mean Squared Error:', mse)
    print('Mean Absolute Error:', mae)
    print('R^2 Score:', r2)


In [180]:
def Create_df_one_gene(drug_df_for_model):
    # Extract the column names (excluding the label column)
    column_names = drug_df_for_model.columns[:-1]

    # Create a new DataFrame with zeros
    df_one_gene = pd.DataFrame(0, index=np.arange(len(column_names)), columns=column_names)

    # Set the value at the corresponding index position in each row
    for i in range(len(column_names)):
        df_one_gene.iloc[i, i] = 1

    # Display the new DataFrame
    return df_one_gene

In [181]:
def PredR_Antibiotic(drug):
    print('************************\n', drug)
    
    drug_df,drug_genes =   gene_per_drug(drug)
    drug_df_for_model = create_gene_df(drug_df, drug_genes)
    y_test, predictions, model = Model(drug_df_for_model)
    evaluation(y_test, predictions, drug)
    df_gene = Create_df_one_gene(drug_df_for_model)
    pred = model.predict(df_gene)
    for i, col in enumerate(df_gene.columns):
        gene_antibiotic_df.loc[col, drug] = pred[i]

In [182]:
antibiotic_one_sample = []
for antibiotic in unique_all_antibiotics:
    if len(df[df['drug'] == antibiotic]) == 1:
        antibiotic_one_sample.append(antibiotic)
        continue
    PredR_Antibiotic(antibiotic)

************************
 chloramphenicol
Epoch 1/10
307/307 [==============================] - 0s 1ms/step - loss: 0.1426 - mse: 0.0345
Epoch 2/10
307/307 [==============================] - 0s 1ms/step - loss: 0.0531 - mse: 0.0066
Epoch 3/10
307/307 [==============================] - 0s 1ms/step - loss: 0.0473 - mse: 0.0053
Epoch 4/10
307/307 [==============================] - 0s 1ms/step - loss: 0.0444 - mse: 0.0047
Epoch 5/10
307/307 [==============================] - 0s 1ms/step - loss: 0.0433 - mse: 0.0046
Epoch 6/10
307/307 [==============================] - 0s 1ms/step - loss: 0.0420 - mse: 0.0043
Epoch 7/10
307/307 [==============================] - 0s 949us/step - loss: 0.0415 - mse: 0.0043
Epoch 8/10
307/307 [==============================] - 0s 880us/step - loss: 0.0412 - mse: 0.0042
Epoch 9/10
307/307 [==============================] - 0s 1ms/step - loss: 0.0405 - mse: 0.0041
Epoch 10/10
77/77 [==============================] - 0s 684us/step - loss: 0.0631 - mse: 0.0078
Los

21/21 [==============================] - 0s 898us/step - loss: 0.1201 - mse: 0.0334
Epoch 6/10
21/21 [==============================] - 0s 898us/step - loss: 0.1039 - mse: 0.0300
Epoch 7/10
21/21 [==============================] - 0s 909us/step - loss: 0.0924 - mse: 0.0276
Epoch 8/10
21/21 [==============================] - 0s 906us/step - loss: 0.0822 - mse: 0.0250
Epoch 9/10
21/21 [==============================] - 0s 872us/step - loss: 0.0751 - mse: 0.0224
Epoch 10/10
6/6 [==============================] - 0s 998us/step - loss: 0.0557 - mse: 0.0099
Loss: 0.055715397000312805
Mean Squared Error: 0.009895863011479378
len of pred:  (168, 1) y_test:  (168,)
Mean Squared Error: 0.009895862792818939
Mean Absolute Error: 0.023048485401513323
R^2 Score: 0.47671975369644637
************************
 piperacillin
Epoch 1/10
2/2 [==============================] - 0s 999us/step - loss: 0.7483 - mse: 0.2565
Epoch 2/10
2/2 [==============================] - 0s 1ms/step - loss: 0.7073 - mse: 0.236

38/38 [==============================] - 0s 1ms/step - loss: 0.1091 - mse: 0.0149
Epoch 9/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1044 - mse: 0.0138
Epoch 10/10
10/10 [==============================] - 0s 970us/step - loss: 0.2458 - mse: 0.0547
Loss: 0.2458273470401764
Mean Squared Error: 0.0546860471367836
len of pred:  (303, 1) y_test:  (303,)
Mean Squared Error: 0.05468604532979819
Mean Absolute Error: 0.09365488871162747
R^2 Score: 0.7560116080338505
************************
 clindamycin
Epoch 1/10
92/92 [==============================] - 0s 1ms/step - loss: 0.3998 - mse: 0.1232
Epoch 2/10
92/92 [==============================] - 0s 1ms/step - loss: 0.1705 - mse: 0.0456
Epoch 3/10
92/92 [==============================] - 0s 1ms/step - loss: 0.1056 - mse: 0.0230
Epoch 4/10
92/92 [==============================] - 0s 1ms/step - loss: 0.0764 - mse: 0.0140
Epoch 5/10
92/92 [==============================] - 0s 996us/step - loss: 0.0718 - mse: 0.0129
Epoch 6/10

************************
 cefalexin
Epoch 1/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6686 - mse: 0.2377
Epoch 2/10
1/1 [==============================] - 0s 997us/step - loss: 0.6542 - mse: 0.2306
Epoch 3/10
1/1 [==============================] - 0s 998us/step - loss: 0.6416 - mse: 0.2243
Epoch 4/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6312 - mse: 0.2191
Epoch 5/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6235 - mse: 0.2153
Epoch 6/10
1/1 [==============================] - 0s 998us/step - loss: 0.6164 - mse: 0.2119
Epoch 7/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6051 - mse: 0.2063
Epoch 8/10
1/1 [==============================] - 0s 997us/step - loss: 0.5923 - mse: 0.1999
Epoch 9/10
1/1 [==============================] - 0s 996us/step - loss: 0.5780 - mse: 0.1929
Epoch 10/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6937 - mse: 2.5285e-04
Loss: 0.6936531662940979
Mean Squared E

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:589: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


...

Epoch 1/10
4/4 [==============================] - 0s 998us/step - loss: 0.7406 - mse: 0.2736
Epoch 2/10
4/4 [==============================] - 0s 1ms/step - loss: 0.6865 - mse: 0.2467
Epoch 3/10
4/4 [==============================] - 0s 998us/step - loss: 0.6603 - mse: 0.2336
Epoch 4/10
4/4 [==============================] - 0s 1ms/step - loss: 0.6404 - mse: 0.2237
Epoch 5/10
4/4 [==============================] - 0s 1ms/step - loss: 0.6168 - mse: 0.2120
Epoch 6/10
4/4 [==============================] - 0s 859us/step - loss: 0.5878 - mse: 0.1978
Epoch 7/10
4/4 [==============================] - 0s 1ms/step - loss: 0.5528 - mse: 0.1810
Epoch 8/10
4/4 [==============================] - 0s 997us/step - loss: 0.5097 - mse: 0.1607
Epoch 9/10
4/4 [==============================] - 0s 855us/step - loss: 0.4649 - mse: 0.1406
Epoch 10/10
1/1 [==============================] - 0s 968us/step - loss: 0.4675 - mse: 0.1339
Loss: 0.4674932062625885
Mean Squared Error: 0.13394354283809662
len of pred:

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:589: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Epoch 1/10
5/5 [==============================] - 0s 998us/step - loss: 0.7187 - mse: 0.2627
Epoch 2/10
5/5 [==============================] - 0s 889us/step - loss: 0.6468 - mse: 0.2269
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: 0.6085 - mse: 0.2078
Epoch 4/10
5/5 [==============================] - 0s 808us/step - loss: 0.5769 - mse: 0.1923
Epoch 5/10
5/5 [==============================] - 0s 999us/step - loss: 0.5359 - mse: 0.1724
Epoch 6/10
5/5 [==============================] - 0s 990us/step - loss: 0.4830 - mse: 0.1474
Epoch 7/10
5/5 [==============================] - 0s 997us/step - loss: 0.4217 - mse: 0.1198
Epoch 8/10
5/5 [==============================] - 0s 806us/step - loss: 0.3551 - mse: 0.0916
Epoch 9/10
5/5 [==============================] - 0s 997us/step - loss: 0.2847 - mse: 0.0648
Epoch 10/10
2/2 [==============================] - 0s 997us/step - loss: 0.1834 - mse: 0.0322
Loss: 0.18343107402324677
Mean Squared Error: 0.0322054959833622
len of

3/3 [==============================] - 0s 1ms/step - loss: 0.6936 - mse: 2.2386e-04
Epoch 5/10
3/3 [==============================] - 0s 993us/step - loss: 0.6935 - mse: 1.7217e-04
Epoch 6/10
3/3 [==============================] - 0s 661us/step - loss: 0.6934 - mse: 1.2251e-04
Epoch 7/10
3/3 [==============================] - 0s 997us/step - loss: 0.6933 - mse: 8.5947e-05
Epoch 8/10
3/3 [==============================] - 0s 665us/step - loss: 0.6933 - mse: 5.8829e-05
Epoch 9/10
3/3 [==============================] - 0s 997us/step - loss: 0.6932 - mse: 4.1413e-05
Epoch 10/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6932 - mse: 2.4872e-05
Loss: 0.6931968927383423
Mean Squared Error: 2.4871527784853242e-05
len of pred:  (17, 1) y_test:  (17,)
Mean Squared Error: 2.4871526399242663e-05
Mean Absolute Error: 0.003312133690890144
R^2 Score: 0.0
************************
 delafloxacin
Epoch 1/10
2/2 [==============================] - 0s 998us/step - loss: 0.7093 - mse: 0.258

************************
 cefiderocol
Epoch 1/10
1/1 [==============================] - 0s 1ms/step - loss: 0.8311 - mse: 0.3009
Epoch 2/10
1/1 [==============================] - 0s 999us/step - loss: 0.7956 - mse: 0.2838
Epoch 3/10
1/1 [==============================] - 0s 999us/step - loss: 0.7665 - mse: 0.2696
Epoch 4/10
1/1 [==============================] - 0s 959us/step - loss: 0.7414 - mse: 0.2573
Epoch 5/10
1/1 [==============================] - 0s 957us/step - loss: 0.7214 - mse: 0.2474
Epoch 6/10
1/1 [==============================] - 0s 1ms/step - loss: 0.7110 - mse: 0.2422
Epoch 7/10
1/1 [==============================] - 0s 0s/step - loss: 0.7039 - mse: 0.2387
Epoch 8/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6984 - mse: 0.2360
Epoch 9/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6935 - mse: 0.2336
Epoch 10/10
1/1 [==============================] - 0s 1ms/step - loss: 0.7283 - mse: 0.2675
Loss: 0.7282583713531494
Mean Squared Error:

In [183]:
antibiotic_one_sample

['dicloxacillin',
 'fosfomycin-glucose-6-phosphate',
 'fidaxomicin',
 'apramycin',
 'cephalexin',
 'ceftizoxime',
 'surotomycin',
 'rifaximin',
 'pefloxacin',
 'mecillinam']

In [184]:
gene_antibiotic_df

,chloramphenicol,dicloxacillin,ciprofloxacin,ceftiofur,fosfomycin-glucose-6-phosphate,amoxicillin-clavulanic acid,benzylpenicillin,metronidazole,linezolid,piperacillin,...,trimethoprim-sulfamethoxazole,aztreonam,norfloxacin,neomycin,Imipenem-EDTA-PA,delafloxacin,zoliflodacin,vancomycin,ertapenem,cefiderocol
qnrB48,-1.000000,-1,0.039699,-1.000000,-1,-1.000000,-1.0,-1.0,-1.000000,-1.000000,...,0.041179,0.282364,-1.000000,-1.000000,0.501277,0.519180,-1.0,-1.000000,0.655702,0.504569
blaADC-155,-1.000000,-1,0.006509,-1.000000,-1,-1.000000,-1.0,-1.0,-1.000000,-1.000000,...,0.004928,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000
oqxB19,0.017815,-1,0.022884,0.121675,-1,0.017955,-1.0,-1.0,0.058612,0.567754,...,0.111477,0.136410,0.426273,0.533598,0.503210,0.510799,-1.0,0.049670,0.116142,-1.000000
blaACT-37,-1.000000,-1,0.004970,-1.000000,-1,0.356468,-1.0,-1.0,-1.000000,-1.000000,...,0.007567,0.681791,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,0.514734,-1.000000
blaTEM-19,-1.000000,-1,0.004653,-1.000000,-1,0.124477,-1.0,-1.0,-1.000000,-1.000000,...,0.010286,0.071376,-1.000000,-1.000000,-1.000000,0.523927,-1.0,-1.000000,0.075586,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
blaOXA-494,-1.000000,-1,0.017026,-1.000000,-1,0.422314,-1.0,-1.0,-1.000000,0.551732,...,0.034108,0.102046,-1.000000,-1.000000,0.501589,0.521212,-1.0,-1.000000,0.093444,0.498074
qnrB38,-1.000000,-1,0.002331,-1.000000,-1,0.693029,-1.0,-1.0,-1.000000,-1.000000,...,0.007532,0.047051,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,0.289587,-1.000000
tmexD3,-1.000000,-1,0.039345,-1.000000,-1,-1.000000,-1.0,-1.0,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000
tet(O),-1.000000,-1,0.009538,-1.000000,-1,0.002270,-1.0,-1.0,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000


### model - accuraccy & MSE

In [32]:
predictions

array([[0.00322947],
       [0.9983469 ],
       [0.00322947],
       ...,
       [0.00265715],
       [0.00217324],
       [0.0024083 ]], dtype=float32)

### Evaluation - continues (MSE)

Mean Squared Error: 0.01619927366086918
Mean Absolute Error: 0.03154599644390347
R^2 Score: 0.8993035344650906


### one examle test

In [17]:
y_test[1]

1.0

In [18]:
model.predict(X_test[1].reshape(1, -1))

array([[0.99743843]], dtype=float32)

### Test for each Gene seperate

,qnrB48,blaADC-155,oqxB19,blaACT-37,blaTEM-19,cmx,rmtF2,blaCTX-M-24,qnrB77,blaOXY-2-6,...,gyrA_D95N,mgrB_M27K,blaCDD,blaOXA-735,pmrB_T140P,blaOXA-494,qnrB38,tmexD3,tet(O),murA_D278E
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [34]:
pred_gene = model.predict(df_one_gene)
data = pd.DataFrame(data=pred_gene)

In [50]:
data

,0
0,0.062887
1,0.002434
2,0.015267
3,0.001975
4,0.001146
...,...
1099,0.016005
1100,0.001224
1101,0.049610
1102,0.007509


In [69]:
genes_list = list(oxacillin_genes)

# Access the element at index 29
gene_at_index = genes_list[1]

print(gene_at_index)

blaADC-155


In [48]:
data = pd.DataFrame(data=pred_gene)
data[data > 0.5].dropna()

,0
8,0.642578
12,0.824058
29,0.997132
42,0.636477
90,0.988201
95,0.888557
146,0.908533
150,0.604366
195,0.978971
227,0.639507


In [36]:
pred_gene.max()

0.9973835

In [37]:
pred_gene.min()

1.4436484e-05

In [ ]:
# going over all the bacteria that are resistant to "chloramphenicol":
for gene_lst in df[(df.drug == "ciprofloxacin") & (df.resistance == 0)]['AMR genotypes'].values:
# going over all the genes of each bacteria :
    for g in gene_lst.split():
        print(g)
        v = np.zeros(X.shape[1])
        v[feature_names.index(g)]=1
        print(classifier.predict_proba([v]))
    print("***************************")